In [14]:
import prepare_nb
import numpy as np
import pandas as pd

from ds_lib.config.paths import RESOURCES_DIR

In [119]:
pd.options.mode.string_storage = "pyarrow"
pd.options.mode.copy_on_write = True
pd.options.mode.__dict__

{'d': {'sim_interactive': False,
  'use_inf_as_na': False,
  'data_manager': 'block',
  'copy_on_write': True,
  'chained_assignment': 'warn',
  'string_storage': 'pyarrow'},
 'prefix': 'mode'}

In [111]:
sample_data_path = RESOURCES_DIR / "sample_data" / '1' / "Customer Call List.xlsx"
df = pd.read_excel(sample_data_path, dtype_backend="pyarrow")

# string stripping
df["Last_Name"] = df["Last_Name"].str.strip(" /_.")

# boolean mapping
cols_to_bool = ['Paying Customer', 'Do_Not_Contact']
for col in cols_to_bool:
    df[col] = df[col].map({"Yes": True, "No": False, "Y": True, "N": False}, na_action="ignore").astype("bool[pyarrow]")

# removing not useful columns
not_useful_cols = ["Not_Useful_Column"]
df.drop(columns=not_useful_cols, inplace=True)

# removing extra characters
df['Phone_Number'] = df['Phone_Number'].str.replace(r'[^a-zA-Z0-9]', "", regex=True)

# dealing with string nan and real nan values
df['Phone_Number'] = np.where(df['Phone_Number'].isin([pd.NA, "Na"]), pd.NA, df["Phone_Number"])

# slicing string and formating
sec1, sec2, sec3 = slice(0, 3), slice(3, 6), slice(6, None)
df['Phone_Number'] = df['Phone_Number'].str[sec1] + "-" + df['Phone_Number'].str[sec2] + "-" + df['Phone_Number'].str[sec3]

# dealig with address columns
df["Address"] = np.where(df["Address"].str.lower().isin([pd.NA, "na", "n/a"]), pd.NA, df["Address"])  # identifying nan values
df[['address_line1', "city", "zip"]] = df["Address"].str.split(",", n=2, expand=True)  # spliting and expanding

# identifying with newly expanded columns' nan values as pd.NA for later processing
for col in ['city', 'zip']:
    df[col].fillna(pd.NA, inplace=True)

# # filling all string columns nan values with empty string
# string_columns = df.select_dtypes("object").columns
# df[string_columns].fillna("", inplace=True)

In [112]:
# null value ratio
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False)

zip                95.238095
city               66.666667
Phone_Number       19.047619
Do_Not_Contact     19.047619
Last_Name           4.761905
Address             4.761905
Paying Customer     4.761905
address_line1       4.761905
CustomerID          0.000000
First_Name          0.000000
dtype: float64

In [113]:
(df.isnull().sum() / df.shape[0] * 100).sort_values(ascending=False)

zip                95.238095
city               66.666667
Phone_Number       19.047619
Do_Not_Contact     19.047619
Last_Name           4.761905
Address             4.761905
Paying Customer     4.761905
address_line1       4.761905
CustomerID          0.000000
First_Name          0.000000
dtype: float64

In [120]:
nvseries = pd.Series((df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False), dtype="double[pyarrow]", name="Null Values")
nvdf = pd.DataFrame(nvseries[nvseries > 0])
nvdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, zip to address_line1
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype          
---  ------       --------------  -----          
 0   Null Values  8 non-null      double[pyarrow]
dtypes: double[pyarrow](1)
memory usage: 129.0+ bytes


In [115]:
contact_df = df[df["Do_Not_Contact"] != True].drop(columns=["Do_Not_Contact"])
contact_df

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Paying Customer,address_line1,city,zip
0,1001,Frodo,Baggins,123-545-5421,"123 Shire Lane, Shire",True,123 Shire Lane,Shire,None
4,1005,Jon,Snow,876-678-3469,123 Dragons Road,True,123 Dragons Road,None,None
6,1007,Jeff,Winger,<NA>,1209 South Street,False,1209 South Street,None,None
7,1008,Sherlock,Holmes,876-678-3469,98 Clue Drive,False,98 Clue Drive,None,None
9,1010,Peter,Parker,123-545-5421,"25th Main Street, New York",True,25th Main Street,New York,None
10,1011,Samwise,Gamgee,<NA>,"612 Shire Lane, Shire",True,612 Shire Lane,Shire,None
12,1013,Don,Draper,123-543-2345,2039 Main Street,True,2039 Main Street,None,None
13,1014,Leslie,Knope,876-678-3469,343 City Parkway,True,343 City Parkway,None,None
14,1015,Toby,Flenderson,304-762-2467,214 HR Avenue,False,214 HR Avenue,None,None
15,1016,Ron,Weasley,123-545-5421,2395 Hogwarts Avenue,False,2395 Hogwarts Avenue,None,None


## np.select vs np.where in pandas

In [116]:
df

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Paying Customer,Do_Not_Contact,address_line1,city,zip
0,1001,Frodo,Baggins,123-545-5421,"123 Shire Lane, Shire",True,False,123 Shire Lane,Shire,None
1,1002,Abed,Nadir,123-643-9775,93 West Main Street,False,True,93 West Main Street,None,None
2,1003,Walter,White,706-695-0392,298 Drugs Driveway,False,<NA>,298 Drugs Driveway,None,None
3,1004,Dwight,Schrute,123-543-2345,"980 Paper Avenue, Pennsylvania, 18503",True,True,980 Paper Avenue,Pennsylvania,18503
4,1005,Jon,Snow,876-678-3469,123 Dragons Road,True,False,123 Dragons Road,None,None
5,1006,Ron,Swanson,304-762-2467,768 City Parkway,True,True,768 City Parkway,None,None
6,1007,Jeff,Winger,<NA>,1209 South Street,False,False,1209 South Street,None,None
7,1008,Sherlock,Holmes,876-678-3469,98 Clue Drive,False,False,98 Clue Drive,None,None
8,1009,Gandalf,<NA>,<NA>,123 Middle Earth,True,<NA>,123 Middle Earth,None,None
9,1010,Peter,Parker,123-545-5421,"25th Main Street, New York",True,False,25th Main Street,New York,None


In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   CustomerID       21 non-null     int64[pyarrow] 
 1   First_Name       21 non-null     string[pyarrow]
 2   Last_Name        20 non-null     string[pyarrow]
 3   Phone_Number     17 non-null     object         
 4   Address          20 non-null     object         
 5   Paying Customer  20 non-null     bool[pyarrow]  
 6   Do_Not_Contact   17 non-null     bool[pyarrow]  
 7   address_line1    20 non-null     object         
 8   city             7 non-null      object         
 9   zip              1 non-null      object         
dtypes: bool[pyarrow](2), int64[pyarrow](1), object(5), string[pyarrow](2)
memory usage: 1.5+ KB


In [71]:
df

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Paying Customer,Do_Not_Contact,address_line1,city,zip
0,1001,Frodo,Baggins,123-545-5421,"123 Shire Lane, Shire",True,False,123 Shire Lane,Shire,<NA>
1,1002,Abed,Nadir,123-643-9775,93 West Main Street,False,True,93 West Main Street,<NA>,<NA>
2,1003,Walter,White,706-695-0392,298 Drugs Driveway,False,<NA>,298 Drugs Driveway,<NA>,<NA>
3,1004,Dwight,Schrute,123-543-2345,"980 Paper Avenue, Pennsylvania, 18503",True,True,980 Paper Avenue,Pennsylvania,18503
4,1005,Jon,Snow,876-678-3469,123 Dragons Road,True,False,123 Dragons Road,<NA>,<NA>
5,1006,Ron,Swanson,304-762-2467,768 City Parkway,True,True,768 City Parkway,<NA>,<NA>
6,1007,Jeff,Winger,<NA>,1209 South Street,False,False,1209 South Street,<NA>,<NA>
7,1008,Sherlock,Holmes,876-678-3469,98 Clue Drive,False,False,98 Clue Drive,<NA>,<NA>
8,1009,Gandalf,<NA>,<NA>,123 Middle Earth,True,<NA>,123 Middle Earth,<NA>,<NA>
9,1010,Peter,Parker,123-545-5421,"25th Main Street, New York",True,False,25th Main Street,New York,<NA>


In [100]:
df.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   CustomerID       21 non-null     int64[pyarrow] 
 1   First_Name       21 non-null     string[pyarrow]
 2   Last_Name        20 non-null     string[pyarrow]
 3   Phone_Number     17 non-null     object         
 4   Address          20 non-null     object         
 5   Paying Customer  20 non-null     bool[pyarrow]  
 6   Do_Not_Contact   17 non-null     bool[pyarrow]  
 7   address_line1    20 non-null     object         
 8   city             7 non-null      object         
 9   zip              1 non-null      object         
dtypes: bool[pyarrow](2), int64[pyarrow](1), object(5), string[pyarrow](2)
memory usage: 1.5+ KB


In [108]:
df[df.select_dtypes("object").columns] = df.select_dtypes("object").astype("string")

In [126]:
df.select_dtypes("object").astype("string").dtypes

Phone_Number     string[pyarrow]
Address          string[pyarrow]
address_line1    string[pyarrow]
city             string[pyarrow]
zip              string[pyarrow]
dtype: object

In [128]:
df.describe(include="all")

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Paying Customer,Do_Not_Contact,address_line1,city,zip
count,21.0,21,20,17,20,20,17,20,7,1
unique,<NA>,19,19,6,19,2,2,19,4,1
top,<NA>,Ron,Skywalker,876-678-3469,"910 Tatooine Road, Tatooine",True,False,910 Tatooine Road,Shire,18503
freq,<NA>,2,2,5,2,14,13,2,2,1
mean,1010.952381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,6.127611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1006.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1011.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Paying Customer,Do_Not_Contact
0,1001,Frodo,Baggins,1235455421,"123 Shire Lane, Shire",True,False
1,1002,Abed,Nadir,1236439775,93 West Main Street,False,True
2,1003,Walter,White,7066950392,298 Drugs Driveway,False,<NA>
3,1004,Dwight,Schrute,1235432345,"980 Paper Avenue, Pennsylvania, 18503",True,True
4,1005,Jon,Snow,8766783469,123 Dragons Road,True,False
5,1006,Ron,Swanson,3047622467,768 City Parkway,True,True
6,1007,Jeff,Winger,<NA>,1209 South Street,False,False
7,1008,Sherlock,Holmes,8766783469,98 Clue Drive,False,False
8,1009,Gandalf,<NA>,<NA>,123 Middle Earth,True,<NA>
9,1010,Peter,Parker,1235455421,"25th Main Street, New York",True,False


In [48]:
type(df['Phone_Number'][8])

str

In [49]:
type(df['Phone_Number'][6])

pandas._libs.missing.NAType

In [50]:
pd.isna(df['Phone_Number'][6])

True

In [8]:
df.head()

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Paying Customer,Do_Not_Contact
0,1001,Frodo,Baggins,123-545-5421,"123 Shire Lane, Shire",True,False
1,1002,Abed,Nadir,123/643/9775,93 West Main Street,False,True
2,1003,Walter,White,7066950392,298 Drugs Driveway,False,<NA>
3,1004,Dwight,Schrute,123-543-2345,"980 Paper Avenue, Pennsylvania, 18503",True,True
4,1005,Jon,Snow,876|678|3469,123 Dragons Road,True,False


In [16]:
df

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Paying Customer,Do_Not_Contact
0,1001,Frodo,Baggins,1235455421,"123 Shire Lane, Shire",True,False
1,1002,Abed,Nadir,1236439775,93 West Main Street,False,True
2,1003,Walter,White,7066950392,298 Drugs Driveway,False,<NA>
3,1004,Dwight,Schrute,1235432345,"980 Paper Avenue, Pennsylvania, 18503",True,True
4,1005,Jon,Snow,8766783469,123 Dragons Road,True,False
5,1006,Ron,Swanson,3047622467,768 City Parkway,True,True
6,1007,Jeff,Winger,,1209 South Street,False,False
7,1008,Sherlock,Holmes,8766783469,98 Clue Drive,False,False
8,1009,Gandalf,<NA>,,123 Middle Earth,True,<NA>
9,1010,Peter,Parker,1235455421,"25th Main Street, New York",True,False


array(['1235455421', '1236439775', '7066950392', '1235432345',
       '8766783469', '3047622467', '', '8766783469', '', '1235455421', '',
       '7066950392', '1235432345', '8766783469', '3047622467',
       '1235455421', '1236439775', '7066950392', '', '8766783469',
       '8766783469'], dtype=object)

In [51]:
df

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Paying Customer,Do_Not_Contact
0,1001,Frodo,Baggins,123-545-5421,"123 Shire Lane, Shire",True,False
1,1002,Abed,Nadir,123/643/9775,93 West Main Street,False,True
2,1003,Walter,White,7066950392,298 Drugs Driveway,False,<NA>
3,1004,Dwight,Schrute,123-543-2345,"980 Paper Avenue, Pennsylvania, 18503",True,True
4,1005,Jon,Snow,876|678|3469,123 Dragons Road,True,False
5,1006,Ron,Swanson,304-762-2467,768 City Parkway,True,True
6,1007,Jeff,Winger,<NA>,1209 South Street,False,False
7,1008,Sherlock,Holmes,876|678|3469,98 Clue Drive,False,False
8,1009,Gandalf,<NA>,N/a,123 Middle Earth,True,<NA>
9,1010,Peter,Parker,123-545-5421,"25th Main Street, New York",True,False
